### Layers : Fundamental blocks of Neural Network

In [1]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable



### Linear(in_features,out_features,bias)

定義 輸入和輸出層的維度(weight的維度)

In [2]:
myLayer = Linear(in_features=10,out_features=5,bias=True)
inp = Variable(torch.randn(1,10))
myLayer = Linear(in_features=10,out_features=5,bias=True) 
myLayer(inp)

tensor([[ 0.1002,  0.5295, -0.7421,  0.0539,  0.4839]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[-0.1717,  0.2007, -0.0108, -0.1848, -0.2601, -0.3067, -0.0661,  0.0136,
         -0.0843,  0.1784],
        [-0.2098,  0.1581,  0.2436,  0.2145,  0.2301, -0.3115,  0.3028, -0.1433,
          0.1267, -0.1453],
        [-0.1469, -0.0331, -0.1525,  0.0945,  0.2838,  0.1154, -0.2574,  0.1292,
         -0.2024, -0.1400],
        [-0.1087,  0.2370, -0.0389,  0.0721, -0.2889, -0.0405, -0.0537,  0.1994,
         -0.0570,  0.0564],
        [ 0.3056,  0.1281,  0.2068,  0.1502,  0.1815,  0.0490, -0.0391,  0.0099,
         -0.2820, -0.1087]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([ 0.1587,  0.1853, -0.1461,  0.1842,  0.1897], requires_grad=True)

### Stacking Linear layers

建立多層神經層透過迭代

In [5]:
myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

tensor([[-0.3075,  0.1490]], grad_fn=<AddmmBackward>)

### PyTorch Non-linear Activations

使用兩種呼叫激勵函數的不同方式

In [6]:
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
myRelu = ReLU()
myRelu(sample_data)


tensor([[1., 2., 0., 0.]])

In [7]:
import torch.nn as nn
import torch.nn.functional as F
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
f = F.relu(sample_data) # Much simpler.
f

tensor([[1., 2., 0., 0.]])

### Neural Network 

利用類別建立多層神經網路

In [8]:
class MyFirstNetwork(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyFirstNetwork,self).__init__() 
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def layer(self):
        first  = [self.layer1.weight.data,self.layer1.bias.data]
        second = [self.layer2.weight.data,self.layer2.bias.data]
        return first,second
    def forward(self,input): 
        out = self.layer1(input) 
        out = F.relu(out)
        out = self.layer2(out) 
        out = F.softmax(out,dim=1)
        return out

model = MyFirstNetwork(224*224,56*56,2)
print(model)

MyFirstNetwork(
  (layer1): Linear(in_features=50176, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)


### Loss

介紹兩種不同的loss function

In [9]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

分類問題常用的loss function: 交叉熵(cross-entropy)

In [10]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)

### loss(x,class)=−x[class]+log(∑exp(x[j]))

利用上述公式自訂函數與內建函數比較

In [11]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
print(output.data.item())

def CrossEntropyLoss(input,target):
    sum = 0
    for i in range(len(input)):
        tmp = 0
        for j in range(len(input[0])):
            tmp+=np.e**(input[i][j])
        sum+=-input[i][target[i]]+np.log(tmp)
    return float(sum)/len(input)

print(CrossEntropyLoss(input.data.numpy(),target.data.numpy()))

3.2438900470733643
3.243889968753861


### Optimizer

將多個圖形利用自訂的類別建立成一個物件

In [12]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
# ### Optimizer
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(224,224)):
        self.files = glob(root_dir)
        self.size = size
        self.arr = np.array([[1,0],[0,1]],float)
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        input = Variable(torch.from_numpy(np.asarray(Image.open(self.files[idx]).resize((1,224*224)), float)))
        if(idx%2==0):
            target = Variable(torch.from_numpy(self.arr[0]))
        else:
            target = Variable(torch.from_numpy(self.arr[1])) 
        return input,target

dataset=DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/classifier/*.jpg")

### Training

In [13]:
import torch.optim as optim
from time import perf_counter
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001,momentum=0.9)
print("Training......")

index = 0
t1 = perf_counter()
t  = t1
for input, target in dataset:
    data=input[:,:,0].view(1,-1).float()+input[:,:,1].view(1,-1).float()+input[:,:,2].view(1,-1).float()
    output = model(data/(3*255.))
    optimizer.zero_grad()
    loss = loss_fn(output, target.float())
    loss.backward()
    optimizer.step()
    index+=1
    if(perf_counter()-t > 30):
        t = perf_counter()
        print("Completion ratio :",str(index*100.0/60)+"%")
t2  = perf_counter()

print(model)
print("--------------------------------------")
print("layer1 :","\nweight :\n",model.layer()[0][0].numpy(),"\nbias :\n",model.layer()[0][1].numpy())
print("")
print("layer2 :","\nweight :\n",model.layer()[1][0].numpy(),"\nbias :\n",model.layer()[1][1].numpy())
print("time :",str((t2-t1)/60.0)+" min")

print("\ntest.....")
test = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/sample_test/*.jpg")
data1=test[0][0][:,:,0].view(1,-1).float()+test[0][0][:,:,1].view(1,-1).float()+test[0][0][:,:,2].view(1,-1).float()
data2=test[1][0][:,:,0].view(1,-1).float()+test[1][0][:,:,1].view(1,-1).float()+test[1][0][:,:,2].view(1,-1).float()
output1 = model(data1/(3*255.))
output2 = model(data2/(3*255.))
print(output1)
print(output2)

Training......
Completion ratio : 78.33333333333333%
MyFirstNetwork(
  (layer1): Linear(in_features=50176, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)
--------------------------------------
layer1 : 
weight :
 [[-4.0745595e-03 -1.7737264e-03 -3.9952695e-03 ... -1.5223277e-03
  -6.5764575e-04 -5.1447330e-04]
 [ 3.3075418e-03  1.5288470e-03 -2.0334944e-03 ... -1.3887929e-04
  -2.3537036e-04  3.7988727e-03]
 [-3.0632194e-03  2.3871611e-03  1.4698431e-03 ... -2.8518622e-03
  -3.7205589e-03 -3.2363478e-03]
 ...
 [ 1.2598843e-03  2.2627027e-03 -3.6647960e-03 ... -2.0439737e-03
   2.0862338e-03  1.4809379e-04]
 [-2.8133362e-03  1.0379807e-03 -4.0546288e-03 ...  3.1449811e-03
  -1.6703553e-03 -6.4434047e-04]
 [-3.2127697e-05 -1.5742085e-03 -2.8998160e-03 ... -2.0021130e-03
  -2.6831119e-03 -4.2259949e-03]] 
bias :
 [-0.00199949 -0.00023586 -0.00376622 ...  0.00163435 -0.00279396
  0.00033448]

layer2 : 
weight :
 [[ 0.0065551  -0.01773075 -0.